# Jupyter notebook 1.1 (Aux): reajusta classes conforme novo resultado do mapeamento

## Import libraries and modules

In [ ]:
# Import library and some pre-installed modules for the pipeline
import os
import re
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import unicodedata


In [ ]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [ ]:
# Get current working directory
os.getcwd()

In [ ]:
# Célula 1 — parâmetros e mapeamento

# ---------- PARÂMETROS ----------
base_dir = os.getcwd()

# Caminhos conforme sua pasta
PATH_STEPS  = os.path.join(base_dir,"results", "1_output_grid", "steps_merged_1to6.gpkg")
PATH_STEP7  = os.path.join(base_dir, "results", "2_toponyms_retrieval", "step7_latest_name_ohsome","step7_consolidado_ohsome.geojson")
LAYER_STEPS = 'steps_merged_1to6'   # GPKG com múltiplas camadas
#PATH_STEPS  = os.path.join(base_dir,"results","0_tests_output","1_output_grid_tests", "amostras_steps1e6_reclass", "steps_merged_1to6_original_amostra.geojson")
#PATH_STEP7  = os.path.join(base_dir,"results","0_tests_output","1_output_grid_tests", "amostras_steps1e6_reclass", "step7_consolidado_ohsome_amostra.geojson")
#LAYER_STEPS = None   # GPKG com múltiplas camadas


# ---------- MAPEAMENTO ----------
# Observação: a regra de PRAÇA e CAMPÇO-QUADRA exige confirmação no nome (needs_praca_name=True; needs_campo_quadra_name=True)
MAPPING = [
    # Praças (antes em edif_constr_lazer → agora cbge_praca) – confirmar no name
    {"tag": "leisure", "value": "park", "old_class": "edif_constr_lazer", "new_class": "cbge_praca", "needs_praca_name": True},

    # Parques (antes em edif_constr_lazer → agora cbge_area_verde)
    {"tag": "leisure", "value": "park", "old_class": "edif_constr_lazer", "new_class": "cbge_area_verde"},

    # Campos/quadras (antes em edif_constr_lazer → agora laz_campo_quadra)
    {"tag": "leisure", "value": "sports_centre", "old_class": "edif_constr_lazer", "new_class": "laz_campo_quadra", "needs_campo_quadra_name":True},
    {"tag": "leisure", "value": "park", "old_class": "edif_constr_lazer", "new_class": "laz_campo_quadra", "needs_campo_quadra_name":True},
    {"tag": "leisure", "value": "stadium", "old_class": "edif_constr_lazer", "new_class": "laz_campo_quadra", "needs_campo_quadra_name":True},

    # Comércio e serviços (antes em edif_saude → agora edif_comerc_serv)
    {"tag": "amenity",    "value": "doctors",  "old_class": "edif_saude", "new_class": "edif_comerc_serv"},
    {"tag": "amenity",    "value": "dentist",  "old_class": "edif_saude", "new_class": "edif_comerc_serv"},
    {"tag": "healthcare", "value": "*",        "old_class": "edif_saude", "new_class": "edif_comerc_serv"},
]

# Sufixos válidos por step
SUFFS_BY_STEP = {
    1: ["total_count", "name_count", "name_ratio"],
    2: ["total_contribs", "name_contribs"],
    4: ["name_tagchange"],
    5: ["users_name"],
    6: [
        "contribuicoes_finais",
        "sigmoid_rmse", "sigmoid_pct_erro",
        "sigmoid_a", "sigmoid_b", "sigmoid_c", "sigmoid_d",
        "inflexao_idx", "inflexao_data",
        "sigmoid_fit_overflow",
        "dias_desde_inflexao",
    ],
}
OBJECT_SUFFS = {"inflexao_data", "sigmoid_fit_overflow"}

# --- Regex robusto para capturar (step, classe, sufixo)
ALL_SUFFIXES = sorted({s for L in SUFFS_BY_STEP.values() for s in L})
SUFFIX_ALT   = "|".join(map(re.escape, ALL_SUFFIXES))
SUFFIX_RE    = re.compile(rf"^step(\d+)_consolidado_(.+?)_({SUFFIX_ALT})$")
PRACA_TOKENS = ["praça", "praca", "square", "largo"]

print("[OK] Parâmetros e mapeamento carregados.")

In [ ]:
# Célula 2 — Funções auxiliares (new)
# ===========================
# Célula 2 — Funções auxiliares (atualizada c/ checagem por nome para PRAÇAS e CAMPOS/QUADRAS)
# ===========================

# Regex geral para colunas step*
SUFFIX_RE = re.compile(r"^step(\d+)_consolidado_(.+)_(.+)$")

def load_layer(path, layer=None):
    """Lê GPKG/GeoJSON. Se layer=None e houver múltiplas camadas no GPKG, gera erro pedindo LAYER_STEPS."""
    p = Path(path)
    if p.suffix.lower() == ".gpkg":
        if layer:
            return gpd.read_file(path, layer=layer)
        else:
            import fiona
            layers = fiona.listlayers(path)
            if len(layers) != 1:
                raise ValueError(f"Arquivo {path} contém {len(layers)} camadas. Defina LAYER_STEPS.")
            return gpd.read_file(path, layer=layers[0])
    else:
        return gpd.read_file(path)

def detect_old_classes(df):
    """Detecta classes existentes olhando as colunas step1_consolidado_*_total_count."""
    classes = []
    for c in df.columns:
        m = re.match(r"^step1_consolidado_(.+?)_total_count$", c)
        if m:
            classes.append(m.group(1))
    return sorted(classes)

def report_misplaced(df):
    """Apenas REPORTA colunas possivelmente fora do step esperado (não remove)."""
    misplaced = []
    for c in df.columns:
        m = SUFFIX_RE.match(c)
        if not m:
            continue
        step = int(m.group(1)); suf = m.group(3)
        if suf not in SUFFS_BY_STEP.get(step, []):
            misplaced.append(c)
    if misplaced:
        print(f"[WARN] {len(misplaced)} colunas possivelmente fora do step esperado (não removidas). Ex.: {misplaced[:8]}")
    else:
        print("[OK] Nenhuma coluna aparentemente fora do step esperado.")
    return misplaced

def ensure_cols(df, classes_to_add):
    """
    Cria TODAS as colunas novas que ainda não existem (todas de uma vez, para evitar fragmentação).
    Para OBJECT_SUFFS usa dtype object, senão float64 zerado.
    """
    new_cols = []
    for step in [1, 2, 4, 5, 6]:
        for cls in classes_to_add:
            for suf in SUFFS_BY_STEP[step]:
                col = f"step{step}_consolidado_{cls}_{suf}"
                if col not in df.columns:
                    new_cols.append(col)

    if not new_cols:
        print("[INFO] Nenhuma nova coluna necessária.")
        return df, []

    data = {}
    for col in new_cols:
        suf = col.rsplit("_", 1)[-1]
        if suf in OBJECT_SUFFS:
            data[col] = pd.Series("", index=df.index, dtype="object")
        else:
            data[col] = pd.Series(0.0, index=df.index, dtype="float64")

    df = pd.concat([df, pd.DataFrame(data)], axis=1)
    print(f"[OK] Criadas {len(new_cols)} novas colunas (todas de uma vez).")
    return df, new_cols

def coerce_numeric_block(df, cols):
    """
    Converte para float64 apenas as colunas pedidas, preservando colunas duplicadas
    (mesmo nome) via atribuição por posição.
    """
    # mantém ordem e remove repetidos em 'cols'
    seen = set()
    unique_cols = []
    for c in cols:
        if c not in seen:
            unique_cols.append(c)
            seen.add(c)

    total_conv = 0
    for col in unique_cols:
        # todas as posições onde o nome da coluna aparece (lida com duplicadas)
        pos = np.where(df.columns.values == col)[0]
        if len(pos) == 0:
            continue

        # sub-dataframe só com essas posições (sempre DataFrame, mesmo se 1 col)
        sub = df.iloc[:, pos]
        conv = sub.apply(pd.to_numeric, errors="coerce").astype("float64").fillna(0.0)

        # atribui de volta por posição (mesmo shape)
        df.iloc[:, pos] = conv.values
        total_conv += conv.shape[1]

    print(f"[OK] Convertidas {total_conv} colunas (contando duplicadas) para float64.")
    return df

# ----------------------------
# Normalização e checagem por nome
# ----------------------------
def _norm(s):
    """Remove acentos e coloca em minúsculas; retorna '' se não for string."""
    if not isinstance(s, str):
        return ""
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return s.lower().strip()

PRACA_TOKENS = {"praca", "praça", "parque praça"}
CAMPO_QUADRA_TOKENS = {
    "campo", "quadra", "poliesportivo", "poliesportiva"}

def is_praca_name(val):
    s = _norm(val)
    return any(tok in s for tok in PRACA_TOKENS)

def is_campo_quadra_name(val):
    s = _norm(val)
    return any(tok in s for tok in CAMPO_QUADRA_TOKENS)

def select_matches(pontos_sj, rule):
    """
    Seleciona linhas de pontos livres (__free__=True) que batem tag/value e (se exigido) nome de praça/campo-quadra.
    Retorna (agg_por_célula, rowids_usados).
    """
    tag, val = rule["tag"], rule["value"]
    m = (pontos_sj["__free__"] == True) & (pontos_sj["tag"] == tag)
    if val != "*":
        m &= (pontos_sj["value"] == val)

    # Confirmações via nome:
    if rule.get("needs_praca_name") or rule.get("new_class") == "cbge_praca":
        m &= pontos_sj["name"].apply(is_praca_name)

    if rule.get("needs_campo_quadra_name") or rule.get("new_class") == "laz_campo_quadra":
        m &= pontos_sj["name"].apply(is_campo_quadra_name)

    sub = pontos_sj.loc[m, ["__rowid__", "idx_steps", "name"]].copy()
    if sub.empty:
        return (pd.DataFrame(columns=["idx_steps", "to_transfer", "to_transfer_name"])
                  .astype({"idx_steps": "int64", "to_transfer": "float64", "to_transfer_name": "float64"}), [])

    sub["has_name"] = sub["name"].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)
    agg = sub.groupby("idx_steps").agg(to_transfer=("name", "size"),
                                       to_transfer_name=("has_name", "sum")).astype("float64")
    rowids = sub["__rowid__"].unique().tolist()
    return agg.reset_index(), rowids

def step_key(c: str):
    """Ordenação estável: (step, classe, posição do sufixo naquele step). Se não casar, empurra pro fim."""
    m = SUFFIX_RE.match(c)
    if not m:
        return (999, c, 999)
    step = int(m.group(1)); cls = m.group(2); suf = m.group(3)
    try:
        suf_idx = SUFFS_BY_STEP[step].index(suf)
    except ValueError:
        suf_idx = 999
    return (step, cls, suf_idx)

print("[OK] Funções auxiliares carregadas.")

In [ ]:
# Célula 3 — Leitura, CRS e sjoin

# 1) Ler dados
gdf_steps = load_layer(PATH_STEPS, LAYER_STEPS)
gdf_pts   = load_layer(PATH_STEP7, None)

print(f"[INFO] gdf_steps: {len(gdf_steps)} feições | CRS: {gdf_steps.crs}")
print(f"[INFO] gdf_pts  : {len(gdf_pts)} feições | CRS: {gdf_pts.crs}")

# 2) Alinhar CRS e sjoin (pontos em células)
if gdf_pts.crs != gdf_steps.crs:
    gdf_pts = gdf_pts.to_crs(gdf_steps.crs)
    print("[OK] CRS de gdf_pts alinhado ao de gdf_steps.")

gdf_steps = gdf_steps.reset_index(drop=True).copy()
gdf_steps["__idx_steps__"] = gdf_steps.index

# preserva um "rowid" próprio dos pontos para poder 'consumir' depois
pontos = gdf_pts.reset_index(drop=True).copy()
pontos["__rowid__"] = pontos.index

for col_need in ["tag", "value", "name"]:
    if col_need not in pontos.columns:
        pontos[col_need] = ""

sj = gpd.sjoin(pontos, gdf_steps[["__idx_steps__", "geometry"]],
               predicate="intersects", how="inner")

# renomeia index_right -> idx_steps
if "index_right" in sj.columns:
    sj = sj.rename(columns={"index_right": "idx_steps"})
elif "__idx_steps__" in sj.columns:
    sj = sj.rename(columns={"__idx_steps__": "idx_steps"})

pontos_sj = sj[["__rowid__", "idx_steps", "tag", "value", "name"]].copy()
pontos_sj["idx_steps"] = pontos_sj["idx_steps"].astype(int)
pontos_sj["__free__"] = True  # ainda não usados

print(f"[OK] sjoin: {len(pontos_sj)} correspondências ponto→célula.")
display(pontos_sj.head())


In [ ]:
# Célula 4 — Dtypes + criação de colunas novas

# Guardar ordem ORIGINAL de colunas, para restaurar depois e só anexar novas ao final
original_cols = gdf_steps.columns.tolist()

# 1) Report apenas (não remove)
_ = report_misplaced(gdf_steps)

# 2) Classes detectadas no arquivo e classes novas do mapeamento
old_classes = detect_old_classes(gdf_steps)
new_classes = sorted({r["new_class"] for r in MAPPING})
print(f"[INFO] Classes antigas detectadas (step1_*_total_count): {old_classes}")
print(f"[INFO] Novas classes do mapeamento: {new_classes}")

# 3) Garantir colunas das novas classes (tudo de uma vez)
gdf_steps, added_cols = ensure_cols(gdf_steps, new_classes)

# 4) Coagir numéricos necessários das classes antigas (somente as que existirem)
numeric_needed = []
for cls in old_classes:
    numeric_needed += [
        f"step1_consolidado_{cls}_total_count",
        f"step1_consolidado_{cls}_name_count",
        f"step1_consolidado_{cls}_name_ratio",
        f"step2_consolidado_{cls}_total_contribs",
        f"step2_consolidado_{cls}_name_contribs",
        f"step4_consolidado_{cls}_name_tagchange",
        f"step5_consolidado_{cls}_users_name",
        f"step6_consolidado_{cls}_contribuicoes_finais",
        f"step6_consolidado_{cls}_sigmoid_rmse",
        f"step6_consolidado_{cls}_sigmoid_pct_erro",
        f"step6_consolidado_{cls}_sigmoid_a",
        f"step6_consolidado_{cls}_sigmoid_b",
        f"step6_consolidado_{cls}_sigmoid_c",
        f"step6_consolidado_{cls}_sigmoid_d",
        f"step6_consolidado_{cls}_inflexao_idx",
        f"step6_consolidado_{cls}_dias_desde_inflexao",
    ]
gdf_steps = coerce_numeric_block(gdf_steps, [c for c in numeric_needed if c in gdf_steps.columns])

print(f"[OK] Preparação concluída. Colunas novas criadas: {len(added_cols)}")


In [ ]:
# Célula 5 — Deltas e aplicação das regras

# 1) Montar tabela delta só com colunas que existem (numéricas)
delta_cols = []
all_classes_for_scan = sorted(set(detect_old_classes(gdf_steps) + new_classes))
for cls in all_classes_for_scan:
    for step, suffs in SUFFS_BY_STEP.items():
        for suf in suffs:
            if suf in OBJECT_SUFFS:
                continue  # delta só para numéricas
            col = f"step{step}_consolidado_{cls}_{suf}"
            if col in gdf_steps.columns:
                delta_cols.append(col)
delta_cols = sorted(set(delta_cols))

delta = pd.DataFrame(0.0, index=gdf_steps.index, columns=delta_cols)
print(f"[OK] Tabela delta criada com {len(delta_cols)} colunas numéricas.")

# 2) Aplicar regras, consumindo pontos já usados para evitar dupla contagem
for rule in MAPPING:
    old_cls = rule["old_class"]
    new_cls = rule["new_class"]

    # step1 'old' precisa existir, senão não tem o que transferir desta classe
    need1 = f"step1_consolidado_{old_cls}_total_count"
    need2 = f"step1_consolidado_{old_cls}_name_count"
    if not (need1 in gdf_steps.columns and need2 in gdf_steps.columns):
        sample_step1_cols = [c for c in gdf_steps.columns if c.startswith("step1_consolidado_")][:10]
        print(f"[SKIP] Classe antiga '{old_cls}' sem colunas do step1.\n"
              f"       Esperadas: {need1}, {need2}\n"
              f"       Amostra de step1 no arquivo: {sample_step1_cols}")
        continue

    agg, rowids = select_matches(pontos_sj, rule)
    if agg.empty:
        print(f"[INFO] Nenhuma correspondência para {old_cls} → {new_cls}")
        continue

    idxs = agg["idx_steps"].astype(int).values
    t  = pd.Series(agg["to_transfer"].values,       index=idxs).astype(float)
    tn = pd.Series(agg["to_transfer_name"].values,  index=idxs).astype(float)

    # frações de transferência (evita div/0)
    tot_old  = pd.to_numeric(gdf_steps.loc[t.index, f"step1_consolidado_{old_cls}_total_count"], errors="coerce").fillna(0.0)
    name_old = pd.to_numeric(gdf_steps.loc[t.index, f"step1_consolidado_{old_cls}_name_count"],  errors="coerce").fillna(0.0)

    with np.errstate(divide='ignore', invalid='ignore'):
        frac_total = (t / tot_old.replace(0, np.nan)).fillna(0.0).clip(0, 1)
        frac_name  = (tn / name_old.replace(0, np.nan)).fillna(0.0).clip(0, 1)

    # ===== STEP 1 =====
    c_old = f"step1_consolidado_{old_cls}_total_count"
    c_new = f"step1_consolidado_{new_cls}_total_count"
    if c_old in delta.columns and c_new in delta.columns:
        delta.loc[t.index, c_old] -= t.values
        delta.loc[t.index, c_new] += t.values

    c_old = f"step1_consolidado_{old_cls}_name_count"
    c_new = f"step1_consolidado_{new_cls}_name_count"
    if c_old in delta.columns and c_new in delta.columns:
        delta.loc[tn.index, c_old] -= tn.values
        delta.loc[tn.index, c_new] += tn.values

    # ===== STEP 2 =====
    for suf, frac in [("total_contribs", frac_total), ("name_contribs", frac_name)]:
        old_col = f"step2_consolidado_{old_cls}_{suf}"
        new_col = f"step2_consolidado_{new_cls}_{suf}"
        if (old_col in delta.columns) and (new_col in delta.columns):
            base = pd.to_numeric(gdf_steps.loc[frac.index, old_col], errors="coerce").fillna(0.0)
            move = (base * frac).values
            delta.loc[frac.index, old_col] -= move
            delta.loc[frac.index, new_col] += move

    # ===== STEP 4 =====
    c4_old = f"step4_consolidado_{old_cls}_name_tagchange"
    c4_new = f"step4_consolidado_{new_cls}_name_tagchange"
    if (c4_old in delta.columns) and (c4_new in delta.columns):
        base = pd.to_numeric(gdf_steps.loc[frac_name.index, c4_old], errors="coerce").fillna(0.0)
        move = (base * frac_name).values
        delta.loc[frac_name.index, c4_old] -= move
        delta.loc[frac_name.index, c4_new] += move

    # ===== STEP 5 =====
    c5_old = f"step5_consolidado_{old_cls}_users_name"
    c5_new = f"step5_consolidado_{new_cls}_users_name"
    if (c5_old in delta.columns) and (c5_new in delta.columns):
        base = pd.to_numeric(gdf_steps.loc[frac_name.index, c5_old], errors="coerce").fillna(0.0)
        move = (base * frac_name).values
        delta.loc[frac_name.index, c5_old] -= move
        delta.loc[frac_name.index, c5_new] += move

    # ===== STEP 6 =====
    # contribuições finais (proporcional ao total)
    c6_old = f"step6_consolidado_{old_cls}_contribuicoes_finais"
    c6_new = f"step6_consolidado_{new_cls}_contribuicoes_finais"
    if (c6_old in delta.columns) and (c6_new in delta.columns):
        base = pd.to_numeric(gdf_steps.loc[frac_total.index, c6_old], errors="coerce").fillna(0.0)
        move = (base * frac_total).values
        delta.loc[frac_total.index, c6_old] -= move
        delta.loc[frac_total.index, c6_new] += move

    # parâmetros sigmoid: copiar 1:1 para as células afetadas (onde houve t>0)
    idx_list = list(t.index)

    sigmoid_float = ["sigmoid_rmse", "sigmoid_pct_erro", "sigmoid_a", "sigmoid_b", "sigmoid_c", "sigmoid_d",
                     "inflexao_idx", "dias_desde_inflexao"]
    for suf in sigmoid_float:
        src = f"step6_consolidado_{old_cls}_{suf}"
        dst = f"step6_consolidado_{new_cls}_{suf}"
        if src in gdf_steps.columns and dst in gdf_steps.columns:
            gdf_steps.loc[idx_list, dst] = pd.to_numeric(gdf_steps.loc[idx_list, src], errors="coerce").astype("float64")

    sigmoid_obj = ["inflexao_data", "sigmoid_fit_overflow"]
    for suf in sigmoid_obj:
        src = f"step6_consolidado_{old_cls}_{suf}"
        dst = f"step6_consolidado_{new_cls}_{suf}"
        if src in gdf_steps.columns and dst in gdf_steps.columns:
            if gdf_steps[dst].dtype != "object":
                gdf_steps[dst] = gdf_steps[dst].astype("object", copy=False)
            gdf_steps.loc[idx_list, dst] = gdf_steps.loc[idx_list, src].astype("object")

    # Consumir pontos usados nesta regra
    if rowids:
        pontos_sj.loc[pontos_sj["__rowid__"].isin(rowids), "__free__"] = False

    print(f"[OK] Regra aplicada: {old_cls} → {new_cls} (células afetadas={len(set(idxs))})")

print("[OK] Todas as regras processadas.")

In [ ]:
# Célula 6 — Aplicar deltas, arredondar discretas e recalcular ratios

if delta.shape[1] > 0:
    # converter bloco numérico antes de somar
    gdf_steps[delta.columns] = pd.to_numeric(gdf_steps[delta.columns].stack(), errors="coerce").unstack().astype("float64").fillna(0.0)
    gdf_steps[delta.columns] = gdf_steps[delta.columns] + delta
    print("[OK] Deltas aplicados.")
else:
    print("[INFO] Nenhum delta numérico a aplicar.")


# 6.0b) Forçar não-negatividade em colunas de contagem
_nonneg_patterns = [
    r"^step1_consolidado_.+_total_count$",
    r"^step1_consolidado_.+_name_count$",
    r"^step2_consolidado_.+_total_contribs$",
    r"^step2_consolidado_.+_name_contribs$",
    r"^step4_consolidado_.+_name_tagchange$",
    r"^step5_consolidado_.+_users_name$",
    r"^step6_consolidado_.+_contribuicoes_finais$",
]

nonneg_cols = [c for c in gdf_steps.columns
               if any(re.match(pat, c) for pat in _nonneg_patterns)]

if nonneg_cols:
    df_nonneg_before = gdf_steps[nonneg_cols].apply(pd.to_numeric, errors="coerce")
    # clip para >= 0
    df_nonneg_after = df_nonneg_before.clip(lower=0).astype("float64").fillna(0.0)
    gdf_steps[nonneg_cols] = df_nonneg_after
    neg_fixed = int((df_nonneg_before < 0).sum().sum())
    print(f"[OK] Não-negatividade aplicada em {len(nonneg_cols)} colunas (corrigidos {neg_fixed} valores negativos).")
else:
    print("[INFO] Nenhuma coluna de contagem para clip não-negativo.")

# 6.1) Arredondar contagens discretas (floor) para manter números inteiros
#      Exemplos: contribuições (step2) e tagchange (step4)

# Quais padrões arredondar (apenas os pedidos)
_floor_patterns = [
    r"^step1_consolidado_.+_total_count$",
    r"^step1_consolidado_.+_name_count$",
    r"^step2_consolidado_.+_total_contribs$",
    r"^step2_consolidado_.+_name_contribs$",
    r"^step4_consolidado_.+_name_tagchange$",
    r"^step5_consolidado_.+_users_name$",
    r"^step6_consolidado_.+_contribuicoes_finais$",
]

# Seleciona as colunas que batem com os padrões
floor_cols = [c for c in gdf_steps.columns
              if any(re.match(pat, c) for pat in _floor_patterns)]

if floor_cols:
    # Converte para numérico, aplica floor e mantém dtype float64 (ex.: 10.0)
    df_floor = gdf_steps[floor_cols].apply(pd.to_numeric, errors="coerce")
    before   = df_floor.copy()
    df_floor = np.floor(df_floor).fillna(0.0).astype("float64")
    gdf_steps[floor_cols] = df_floor

    # Log opcional: quantas células mudaram
    changed_cells = (before != df_floor).sum().sum()
    print(f"[OK] Arredondadas {len(floor_cols)} colunas discretas (mudanças em {int(changed_cells)} células).")
else:
    print("[INFO] Nenhuma coluna discreta para arredondar (step2/step4).")

# (deixe em seguida o seu passo 7: recálculo dos ratios)

# Recalcular ratios step 1
all_classes_now = sorted(set(detect_old_classes(gdf_steps) + list({r["new_class"] for r in MAPPING})))
recalc = 0
for cls in all_classes_now:
    tot_col   = f"step1_consolidado_{cls}_total_count"
    name_col  = f"step1_consolidado_{cls}_name_count"
    ratio_col = f"step1_consolidado_{cls}_name_ratio"
    if all(c in gdf_steps.columns for c in [tot_col, name_col, ratio_col]):
        tot  = pd.to_numeric(gdf_steps[tot_col], errors="coerce")
        name = pd.to_numeric(gdf_steps[name_col], errors="coerce")
        with np.errstate(divide='ignore', invalid='ignore'):
            ratio = (name / tot.replace(0, np.nan) * 100.0).fillna(0.0)
        gdf_steps[ratio_col] = ratio.astype("float64")
        recalc += 1
print(f"[OK] Ratios recalculados para {recalc} classes.")


In [ ]:
# Célula 7 — Checagens rápidas (opcional)

def sum_step1_totals(df, classes):
    vals = {}
    for cls in classes:
        col = f"step1_consolidado_{cls}_total_count"
        if col in df.columns:
            vals[cls] = float(pd.to_numeric(df[col], errors="coerce").fillna(0.0).sum())
    return vals

classes_check = sorted(set([r["old_class"] for r in MAPPING] + [r["new_class"] for r in MAPPING]))
totais = sum_step1_totals(gdf_steps, classes_check)
print("[INFO] Soma step1_total_count das classes mapeadas:", totais)

# conferir linhas afetadas para uma regra específica
cls_new = "cbge_area_verde"
for col in [f"step1_consolidado_{cls_new}_total_count", f"step1_consolidado_{cls_new}_name_count", f"step1_consolidado_{cls_new}_name_ratio"]:
    if col in gdf_steps.columns:
        print(col, "— valores não nulos:", int((pd.to_numeric(gdf_steps[col], errors="coerce").fillna(0.0) != 0).sum()))


In [ ]:
# Célula 7 — Reordenar colunas e salvar

# Ordenar novas colunas pelo step_key; manter TODAS as colunas originais na MESMA ORDEM
new_cols_sorted = sorted([c for c in gdf_steps.columns if c not in original_cols], key=step_key)
final_order = [c for c in original_cols if c in gdf_steps.columns] + new_cols_sorted
gdf_steps = gdf_steps[final_order].copy()

print(f"[OK] Reordenação concluída. Novas colunas anexadas ao final: {len(new_cols_sorted)}")
print("[INFO] Exemplo das últimas 15 colunas (esperado novas no final):")
print(gdf_steps.columns[-15:].tolist())

In [ ]:
# Célula 8 — Visualização final
display(gdf_steps.head())

In [ ]:
# Celula 9 — salvar (GPKG ou GeoJSON)
base_dir = os.getcwd()
#out_path  = os.path.join(base_dir,"results","0_tests_output","1_output_grid_tests", "amostras_steps1e6_reclass", "steps_merged_1to6_test_reclass.geojson")
out_path = os.path.join(base_dir, "results", "1_output_grid", "steps_merged_1to6_reclass.gpkg")
gdf_steps.to_file(out_path, driver="GPKG", layer="steps_merged_1to6_reclass", index=False)
print(f"[OK] Arquivo salvo em: {out_path}")